**A Detailed Guide to understand the Word Embeddings and Embedding Layer in Keras.**

In this notebook I have explained the keras embedding layer. To do so I have created a sample corpus of just 3 documents and that should be sufficient to explain the working of the keras embedding layer.

Embeddings are useful in a variety of machine learning applications. Because of the fact I have attached many data sources to the kernel where I fell that embeddings and Keras embedding layer may prove to be useful.

Before diving in let us skim through some of the applilcations of the embeddings :



1.   The first application that strikes me is in the Collaborative Filtering based Recommender Systems where we have to create the user embeddings and the movie embeddings by decomposing the utility matrix which contains the user-item ratings.


2. The second use is in the Natural Language Processing and its related applications whre we have to create the word embeddings for all the words present in the documents of our corpus.


Thus the embedding layer in Keras can be used when we want to create the embeddings to embed higher dimensional data into lower dimensional vector space.



**1. Importing Modules**

In [2]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#nltk
import nltk

#stop-words
from nltk.corpus import stopwords
stop_words=set(nltk.corpus.stopwords.words('english'))

# tokenizing
from nltk import word_tokenize,sent_tokenize

#keras
import keras
from keras.preprocessing.text import one_hot,Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense , Flatten ,Embedding,Input
from keras.models import Model

Using TensorFlow backend.

**CREATING SAMPLE CORPUS OF DOCUMENTS ie TEXTS**

In [4]:
sample_text_1="bitty bought a bit of butter"
sample_text_2="but the bit of butter was a bit bitter"
sample_text_3="so she bought some better butter to make the bitter butter better"

corp=[sample_text_1,sample_text_2,sample_text_3]
no_docs=len(corp)

**INTEGER ENCODING ALL THE DOCUMENTS**

After this all the unique words will be reprsented by an integer. For this we are using one_hot function from the Keras. Note that the vocab_size is specified large enough so as to ensure unique integer encoding for each and every word.

**Note one important thing that the integer encoding for the word remains same in different docs. eg 'butter' is denoted by 31 in each and every document.**

In [5]:
vocab_size=50 
encod_corp=[]
for i,doc in enumerate(corp):
    encod_corp.append(one_hot(doc,50))
    print("The encoding for document",i+1," is : ",one_hot(doc,50))

The encoding for document 1  is :  [11, 44, 6, 15, 16, 5]
The encoding for document 2  is :  [29, 25, 15, 16, 5, 39, 6, 15, 25]
The encoding for document 3  is :  [48, 47, 44, 44, 16, 5, 19, 11, 25, 25, 5, 16]


**PADDING THE DOCS (to make very doc of same length)**

The Keras Embedding layer requires all individual documents to be of same length. Hence we wil pad the shorter documents with 0 for now. Therefore now in Keras Embedding layer the 'input_length' will be equal to the length (ie no of words) of the document with maximum length or maximum number of words.

**To pad the shorter documents I am using pad_sequences functon from the Keras library.**

In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
# length of maximum document. will be nedded whenever create embeddings for the words
maxlen=-1
for doc in corp:
    tokens=nltk.word_tokenize(doc)
    if(maxlen<len(tokens)):
        maxlen=len(tokens)
print("The maximum number of words in any document is : ",maxlen)

The maximum number of words in any document is :  12


In [9]:
# now to create embeddings all of our docs need to be of same length. hence we can pad the docs with zeros.
pad_corp=pad_sequences(encod_corp,maxlen=maxlen,padding='post',value=0.0)
print("No of padded documents: ",len(pad_corp))

No of padded documents:  3


In [10]:
for i,doc in enumerate(pad_corp):
     print("The padded encoding for document",i+1," is : ",doc)

The padded encoding for document 1  is :  [11 44  6 15 16  5  0  0  0  0  0  0]
The padded encoding for document 2  is :  [29 25 15 16  5 39  6 15 25  0  0  0]
The padded encoding for document 3  is :  [48 47 44 44 16  5 19 11 25 25  5 16]


**ACTUAL CREATION OF THE EMBEDDINGS using KERAS EMBEDDING LAYER**

Now all the documents are of same length (after padding). And so now we are ready to create and use the embeddings.

**I will embed the words into vectors of 8 dimensions.**

In [11]:
# specifying the input shape
input=Input(shape=(no_docs,maxlen),dtype='float64')

In [12]:
'''
shape of input. 
each document has 12 element or words which is the value of our maxlen variable.

'''
word_input=Input(shape=(maxlen,),dtype='float64')  

# creating the embedding
word_embedding=Embedding(input_dim=vocab_size,output_dim=8,input_length=maxlen)(word_input)

word_vec=Flatten()(word_embedding) # flatten
embed_model =Model([word_input],word_vec) # combining all into a Keras model

PARAMETERS OF THE EMBEDDING LAYER ---

'input_dim' = the vocab size that we will choose. In other words it is the number of unique words in the vocab.

'output_dim' = the number of dimensions we wish to embed into. Each word will be represented by a vector of this much dimensions.

'input_length' = lenght of the maximum document. which is stored in maxlen variable in our case.

In [13]:
embed_model.compile(optimizer=keras.optimizers.Adam(lr=1e-3),loss='binary_crossentropy',metrics=['acc']) 
# compiling the model. parameters can be tuned as always.

In [14]:
print(type(word_embedding))
print(word_embedding)

<class 'tensorflow.python.framework.ops.Tensor'>
Tensor("embedding/embedding_lookup/Identity_1:0", shape=(None, 12, 8), dtype=float32)


In [15]:
print(embed_model.summary()) # summary of the model

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 12)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 12, 8)             400       
_________________________________________________________________
flatten (Flatten)            (None, 96)                0         
Total params: 400
Trainable params: 400
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
embeddings=embed_model.predict(pad_corp) # finally getting the embeddings.

In [17]:
print("Shape of embeddings : ",embeddings.shape)
print(embeddings)

Shape of embeddings :  (3, 96)
[[ 0.02643747 -0.03273996  0.00447986  0.04219189  0.03548241 -0.03581563
  -0.03948783 -0.01811973  0.00977848  0.03345742 -0.01126305  0.02179888
  -0.0214689  -0.02987996 -0.02487725  0.03263691 -0.03453332  0.04029364
   0.01236478 -0.02904075  0.02867818 -0.01142744  0.01930623  0.0459299
  -0.01645188 -0.00654026  0.02132687 -0.02805109  0.03854269  0.0416329
   0.0121117   0.02978028 -0.02626245  0.0153589   0.01248128  0.03462856
   0.04364327 -0.02913158  0.03644614 -0.00456511 -0.00137649  0.03332639
  -0.0328102   0.01769571 -0.03066769 -0.02801114 -0.01526086 -0.02591366
  -0.04280609  0.0265871   0.03912641  0.02125232  0.02215796 -0.03851732
   0.01395937 -0.04208454 -0.04280609  0.0265871   0.03912641  0.02125232
   0.02215796 -0.03851732  0.01395937 -0.04208454 -0.04280609  0.0265871
   0.03912641  0.02125232  0.02215796 -0.03851732  0.01395937 -0.04208454
  -0.04280609  0.0265871   0.03912641  0.02125232  0.02215796 -0.03851732
   0.01395

In [18]:
embeddings=embeddings.reshape(-1,maxlen,8)
print("Shape of embeddings : ",embeddings.shape) 
print(embeddings)

Shape of embeddings :  (3, 12, 8)
[[[ 0.02643747 -0.03273996  0.00447986  0.04219189  0.03548241
   -0.03581563 -0.03948783 -0.01811973]
  [ 0.00977848  0.03345742 -0.01126305  0.02179888 -0.0214689
   -0.02987996 -0.02487725  0.03263691]
  [-0.03453332  0.04029364  0.01236478 -0.02904075  0.02867818
   -0.01142744  0.01930623  0.0459299 ]
  [-0.01645188 -0.00654026  0.02132687 -0.02805109  0.03854269
    0.0416329   0.0121117   0.02978028]
  [-0.02626245  0.0153589   0.01248128  0.03462856  0.04364327
   -0.02913158  0.03644614 -0.00456511]
  [-0.00137649  0.03332639 -0.0328102   0.01769571 -0.03066769
   -0.02801114 -0.01526086 -0.02591366]
  [-0.04280609  0.0265871   0.03912641  0.02125232  0.02215796
   -0.03851732  0.01395937 -0.04208454]
  [-0.04280609  0.0265871   0.03912641  0.02125232  0.02215796
   -0.03851732  0.01395937 -0.04208454]
  [-0.04280609  0.0265871   0.03912641  0.02125232  0.02215796
   -0.03851732  0.01395937 -0.04208454]
  [-0.04280609  0.0265871   0.03912641  

The resulting shape is (3,12,8).

3---> no of documents

12---> each document is made of 12 words which was our maximum length of any document.

& 8---> each word is 8 dimensional.



**GETTING ENCODING FOR A PARTICULAR WORD IN A SPECIFIC DOCUMENT**

In [19]:
for i,doc in enumerate(embeddings):
    for j,word in enumerate(doc):
        print("The encoding for ",j+1,"th word","in",i+1,"th document is : \n\n",word)

The encoding for  1 th word in 1 th document is : 

 [ 0.02643747 -0.03273996  0.00447986  0.04219189  0.03548241 -0.03581563
 -0.03948783 -0.01811973]
The encoding for  2 th word in 1 th document is : 

 [ 0.00977848  0.03345742 -0.01126305  0.02179888 -0.0214689  -0.02987996
 -0.02487725  0.03263691]
The encoding for  3 th word in 1 th document is : 

 [-0.03453332  0.04029364  0.01236478 -0.02904075  0.02867818 -0.01142744
  0.01930623  0.0459299 ]
The encoding for  4 th word in 1 th document is : 

 [-0.01645188 -0.00654026  0.02132687 -0.02805109  0.03854269  0.0416329
  0.0121117   0.02978028]
The encoding for  5 th word in 1 th document is : 

 [-0.02626245  0.0153589   0.01248128  0.03462856  0.04364327 -0.02913158
  0.03644614 -0.00456511]
The encoding for  6 th word in 1 th document is : 

 [-0.00137649  0.03332639 -0.0328102   0.01769571 -0.03066769 -0.02801114
 -0.01526086 -0.02591366]
The encoding for  7 th word in 1 th document is : 

 [-0.04280609  0.0265871   0.03912641

Now this makes it easier to visualize that we have 3(size of corp) documents with each consisting of 12(maxlen) words and each word mapped to a 8-dimensional vector.

Just like above we can now use any other document. We can sent_tokenize the doc into sentences.

Each sentence has a list of words which we will integer encode using the 'one_hot' function as below.

Now each sentence will be having different number of words. So we will need to pad the sequences to the sentence with maximum words.

At this point we are ready to feed the input to Keras Embedding layer as shown above.

'input_dim' = the vocab size that we will choose

'output_dim' = the number of dimensions we wish to embed into

'input_length' = lenght of the maximum document